In [1]:
##For Data consideration see Kaggle data tab.
##Predict for 
#id,date,store_nbr,item_nbr,onpromotion
#125497040,2017-08-16,1,96995,False
#...
#128867503,2017-08-31,54,2134244,False


#Inspired on Kaggle
#(https://www.kaggle.com/tarobxl/how-the-test-set-is-split-lb-0-532)

#import a dataset
import pandas as pd
from datetime import timedelta

dtypes = {'id':'int64', 'item_nbr':'int32', 'store_nbr':'int8'}

train = pd.read_csv('/Users/Koos/Downloads/DataFav/train.csv', usecols=[1,2,3,4], dtype=dtypes, parse_dates=['date'],
                    skiprows=range(1, 101688779) #Skip dates before 2017-01-01
                    )

In [2]:
train.loc[(train.unit_sales<0),'unit_sales'] = 0 # eliminate negatives
train['unit_sales'] =  train['unit_sales'].apply(pd.np.log1p) #logarithm conversion
train['dow'] = train['date'].dt.dayofweek # dow is day of week - KtB


In [3]:
# creating records for all items, in all markets on all dates
# for correct calculation of daily unit sales averages.
u_dates = train.date.unique()
u_stores = train.store_nbr.unique()
u_items = train.item_nbr.unique()
train.set_index(['date', 'store_nbr', 'item_nbr'], inplace=True)
train = train.reindex(
    pd.MultiIndex.from_product(
        (u_dates, u_stores, u_items),
        names=['date','store_nbr','item_nbr']
    )
)

In [5]:
#clean up
del u_dates, u_stores, u_items

In [6]:
# Fill NaNs
train.loc[:, 'unit_sales'].fillna(0, inplace=True)
train.reset_index(inplace=True) # reset index and restoring unique columns  
lastdate = train.iloc[train.shape[0]-1].date

In [7]:
test = pd.read_csv('/Users/Koos/Downloads/DataFav/test.csv', usecols=[0,1,2,3,4], dtype=dtypes, parse_dates=['date'])
test['dow'] = test['date'].dt.dayofweek

In [8]:
ma_dw = train[['item_nbr','store_nbr','dow','unit_sales']].groupby(['item_nbr','store_nbr','dow'])['unit_sales'].mean().to_frame('madw')
ma_dw.reset_index(inplace=True)
ma_wk = ma_dw[['item_nbr','store_nbr','madw']].groupby(['store_nbr', 'item_nbr'])['madw'].mean().to_frame('mawk')
ma_wk.reset_index(inplace=True)

In [9]:
ma_wk.head()

,store_nbr,item_nbr,mawk
0,1,96995,0.844024
1,1,99197,1.016067
2,1,103520,1.083762
3,1,103665,1.289606
4,1,105574,1.814829


In [10]:
#Moving Averages
#.to_frame is a renaming function of pandas

ma_is = train[['item_nbr','store_nbr','unit_sales']].groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais226')
for i in [112,56,28,14,7,3,1]:
    tmp = train[train.date>lastdate-timedelta(int(i))]
    tmpg = tmp.groupby(['item_nbr','store_nbr'])['unit_sales'].mean().to_frame('mais'+str(i))
    ma_is = ma_is.join(tmpg, how='left')


In [16]:
ma_is['mais']=ma_is.median(axis=1)  ##Which median is used here?

In [18]:
ma_is.reset_index(inplace=True) #delete multi-indexer

In [20]:
## make a test set. A lot of NAN's now.

test = pd.merge(test, ma_is, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_wk, how='left', on=['item_nbr','store_nbr'])
test = pd.merge(test, ma_dw, how='left', on=['item_nbr','store_nbr','dow'])

In [21]:
#clean up
del ma_is, ma_wk, ma_dw

,id,date,store_nbr,item_nbr,onpromotion,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais,mawk,madw
0,125497040,2017-08-16,1,96995,False,2,0.094532,0.154255,0.172356,0.295202,0.334438,0.099021,0.000000,0.000000,0.126638,0.844024,0.693147
1,125497041,2017-08-16,1,99197,False,2,0.286959,0.470665,0.397257,0.280721,0.206455,0.156945,0.000000,0.000000,0.243588,1.016067,1.003476
2,125497042,2017-08-16,1,103501,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
3,125497043,2017-08-16,1,103520,False,2,0.735661,0.815118,0.773160,0.726316,0.573577,0.495105,0.231049,0.000000,0.649946,1.083762,1.068721
4,125497044,2017-08-16,1,103665,False,2,1.015079,1.038379,0.996363,1.026335,1.031388,0.980990,0.462098,0.693147,1.005721,1.289606,1.312220
5,125497045,2017-08-16,1,105574,False,2,1.745741,1.721587,1.702692,1.699200,1.629185,1.560437,0.998577,1.609438,1.664193,1.814829,2.134940
6,125497046,2017-08-16,1,105575,False,2,2.264586,2.240521,2.294462,2.306424,2.382527,2.304131,2.123871,2.197225,2.279524,2.283542,2.454617
7,125497047,2017-08-16,1,105576,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
8,125497048,2017-08-16,1,105577,False,2,0.562736,0.598495,0.536972,0.517999,0.474556,0.553029,0.597253,1.098612,0.557882,1.029221,1.160824
9,125497049,2017-08-16,1,105693,False,2,0.170306,0.171770,0.244519,0.276514,0.276514,0.553029,0.693147,0.693147,0.276514,0.796376,0.877450


In [22]:
test['unit_sales'] = test.mais

In [26]:
test_pos = test.loc[pos_idx]
test.loc[pos_idx, 'unit_sales'] = test_pos['mais'] * test_pos['madw'] / test_pos['mawk']
test.loc[:, "unit_sales"].fillna(0, inplace=True)
test['unit_sales'] = test['unit_sales'].apply(pd.np.expm1)

In [27]:
test.loc[test['onpromotion'] == True, 'unit_sales'] = test.loc[test['onpromotion'] == True, 'unit_sales'] * 1.14

In [33]:
test

,id,date,store_nbr,item_nbr,onpromotion,dow,mais226,mais112,mais56,mais28,mais14,mais7,mais3,mais1,mais,mawk,madw,unit_sales
0,125497040,2017-08-16,1,96995,False,2,0.094532,0.154255,0.172356,0.295202,0.334438,0.099021,0.000000,0.000000,0.126638,0.844024,0.693147,0.109601
1,125497041,2017-08-16,1,99197,False,2,0.286959,0.470665,0.397257,0.280721,0.206455,0.156945,0.000000,0.000000,0.243588,1.016067,1.003476,0.271973
2,125497042,2017-08-16,1,103501,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
3,125497043,2017-08-16,1,103520,False,2,0.735661,0.815118,0.773160,0.726316,0.573577,0.495105,0.231049,0.000000,0.649946,1.083762,1.068721,0.898238
4,125497044,2017-08-16,1,103665,False,2,1.015079,1.038379,0.996363,1.026335,1.031388,0.980990,0.462098,0.693147,1.005721,1.289606,1.312220,1.782520
5,125497045,2017-08-16,1,105574,False,2,1.745741,1.721587,1.702692,1.699200,1.629185,1.560437,0.998577,1.609438,1.664193,1.814829,2.134940,6.083250
6,125497046,2017-08-16,1,105575,False,2,2.264586,2.240521,2.294462,2.306424,2.382527,2.304131,2.123871,2.197225,2.279524,2.283542,2.454617,10.591810
7,125497047,2017-08-16,1,105576,False,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000
8,125497048,2017-08-16,1,105577,False,2,0.562736,0.598495,0.536972,0.517999,0.474556,0.553029,0.597253,1.098612,0.557882,1.029221,1.160824,0.876140
9,125497049,2017-08-16,1,105693,False,2,0.170306,0.171770,0.244519,0.276514,0.276514,0.553029,0.693147,0.693147,0.276514,0.796376,0.877450,0.356170


In [34]:
moreresearch = train.loc[(train.unit_sales=0),'unit_sales']

SyntaxError: invalid syntax (<ipython-input-34-4c5a59ed051d>, line 1)

In [31]:
test[['id','unit_sales']].to_csv('ma8dwof.csv.gz', index=False, float_format='%.3f', compression='gzip')